vtensorflow core implementation** of [mriberodiaz/federatedRecsys](https://github.com/mriberodiaz/federatedRecsys/blob/master/utilsMovie.py)

In [37]:
from src.data_examples.ex2_data_loader import ExampleDataLoader
from src.tf_utils import df_to_tfds

import pandas as pd
import tensorflow as tf
import numpy as np

In [38]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [98]:
data_loader.df_ratings


,user_id,movie_id,rating,unix_timestamp
0,195,241,3.0,881250949
1,185,301,3.0,891717742
2,21,376,1.0,878887116
3,243,50,2.0,880606923
4,165,345,1.0,886397596
...,...,...,...,...
99995,879,475,3.0,880175444
99996,715,203,5.0,879795543
99997,275,1089,1.0,874795795
99998,12,224,2.0,882399156


In [52]:
# build sparse matrix tensor from dataframe
def build_sparse_tensor(df: pd.DataFrame, indices_cols: list, value_col: str, dense_shape: tuple):
  indices = df[indices_cols].values
  values = df[value_col].values

  return tf.SparseTensor(
    indices=indices,
    values=values,
    dense_shape=dense_shape
  )


# tf sparse matrix mean squared error
def sparse_mse(sparse, embeddings: list):
  loss = 0

  axis_els = tf.gather(embeddings.pop(0), sparse.indices[:, 0])
  i = 0
  for embedding in embeddings[1:]:
    i += 1 
    axis_els *= tf.gather(embedding, sparse.indices[:, 0])
  sums = tf.reduce_sum(axis_els, axis=1)
  loss = tf.losses.mean_squared_error(sparse.values, sums)
  return loss

In [53]:
rating_sparse = build_sparse_tensor(data_loader.df_ratings, 
  indices_cols=['movie_id', 'user_id'],
  value_col='rating',
  dense_shape=[
    data_loader.df_movies.shape[0],
    data_loader.df_users.shape[0]]
)

rating_sparse

SparseTensor(indices=tf.Tensor(
[[ 241  195]
 [ 301  185]
 [ 376   21]
 ...
 [1089  275]
 [ 224   12]
 [ 202   11]], shape=(100000, 2), dtype=int64), values=tf.Tensor([3. 3. 1. ... 1. 2. 3.], shape=(100000,), dtype=float64), dense_shape=tf.Tensor([1682  943], shape=(2,), dtype=int64))

In [97]:
U = tf.Variable(tf.random.normal([rating_sparse.dense_shape[0], 3], stddev=1.0))
V = tf.Variable(tf.random.normal([rating_sparse.dense_shape[1], 3], stddev=1.0))

embeddings = {
    "user_id": U,
    "movie_id": V
}

__rating_loss = sparse_mse(rating_sparse, embeddings=[U, V])

__rating_loss

<tf.Tensor: shape=(), dtype=float32, numpy=17.05524>

In [ ]:
class CollaborativeFilteringModel:
  def __init__(self, embedding_vars, loss, metrics=None):
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics

    self._embeddings = {i:None for i in self._embedding_vars}
    self._session = None

  def train(
    self,
    n_iter=1000,
    learning_rate=.5
  ):
    return